## This code is uses NeuroKit2 to analyze EDA data for the KPE study
- We take event files and acq files for each subjects
- We preprocess them and generate epoch for each script (i.e. condition)
- We save a csv file of data for each subject for further analysis 

In [41]:
import neurokit2 as nk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob

In [65]:
# set parameters
output_dir = '/media/Data/Lab_Projects/KPE_PTSD_Project/behavioral/results'
ses = '1' # set session
acq_files = glob.glob('/media/Data/Lab_Projects/KPE_PTSD_Project/behavioral/physio_data/raw/kpe*/scan_%s/kpe*_scripts*.acq' %ses)
events_file = glob.glob('/media/Data/Lab_Projects/KPE_PTSD_Project/neuroimaging/KPE_BIDS/condition_files/sub-*_ses-%s.csv' %ses)

In [66]:
acq_files.sort()
events_file.sort() # sorting by sub number

In [67]:
acq_files

['/media/Data/Lab_Projects/KPE_PTSD_Project/behavioral/physio_data/raw/kpe008/scan_1/kpe008.1_scripts_2016-10-24T09_00_11.acq',
 '/media/Data/Lab_Projects/KPE_PTSD_Project/behavioral/physio_data/raw/kpe1223/scan_1/kpe1223.1_scripts_2017-01-30T08_17_08.acq',
 '/media/Data/Lab_Projects/KPE_PTSD_Project/behavioral/physio_data/raw/kpe1253/scan_1/kpe1253.1_scripts_2016-12-12T08_12_57.acq',
 '/media/Data/Lab_Projects/KPE_PTSD_Project/behavioral/physio_data/raw/kpe1263/scan_1/kpe1263.1_scripts_2016-11-28T08_38_41.acq',
 '/media/Data/Lab_Projects/KPE_PTSD_Project/behavioral/physio_data/raw/kpe1293/scan_1/kpe1293.1_scripts_2017-02-27T08_18_18.acq',
 '/media/Data/Lab_Projects/KPE_PTSD_Project/behavioral/physio_data/raw/kpe1307/scan_1/kpe1307.1_scripts_2017-06-05T08_31_00.acq',
 '/media/Data/Lab_Projects/KPE_PTSD_Project/behavioral/physio_data/raw/kpe1315/scan_1/kpe1315.1_scripts_2017-04-10T08_31_02.acq',
 '/media/Data/Lab_Projects/KPE_PTSD_Project/behavioral/physio_data/raw/kpe1322/scan_1/kpe132

In [68]:
len(acq_files)

25

In [69]:
len(events_file)
for acq, file in zip(acq_files, events_file):
    print(file.split('sub-')[1].split('_ses')[0])
    print(f"Acq file {acq.split('kpe')[1].split('/scan')[0]}")
    

008
Acq file 008
1223
Acq file 1223
1253
Acq file 1253
1263
Acq file 1263
1293
Acq file 1293
1307
Acq file 1307
1315
Acq file 1315
1322
Acq file 1322
1339
Acq file 1339
1343
Acq file 1343
1351
Acq file 1351
1356
Acq file 1356
1364
Acq file 1364
1369
Acq file 1369
1387
Acq file 1387
1390
Acq file 1390
1403
Acq file 1403
1419
Acq file 1419
1464
Acq file 1464
1468
Acq file 1468
1480
Acq file 1480
1499
Acq file 1499
1561
Acq file 1561
1573
Acq file 1573
1578
Acq file 1578


In [70]:
# find epochs to match 
# first read condition list from subjects' events file

def readAnalyze_acq(file, ev_file):
    # this function takes the acq file and events file and returns a data frame of 
    # trial type and max EDA
    a = nk.read_acqknowledge(file, sampling_rate=100,
                     resample_method="interpolation", impute_missing=True)
    events_file = pd.read_csv(ev_file, sep='\t')
    condition_list = list(events_file.trial_type)

    # then use the events_find function to determine onset and duration of the events 
    #(here its going from 0 to 5, so threshold is above)
    events = nk.events_find(event_channel=a[0].Script,
                            threshold_keep='above',
                            event_conditions=condition_list)

    # Preprocess the data (filter, find peaks, etc.)
    processed_data, info = nk.bio_process(#ecg=a[0]["ECG100C"],
                                          eda=a[0]["GSR100C"],
                                          keep=a[0]["Script"],
                                          sampling_rate=a[1])


    # create epochs
    epochs = nk.epochs_create(processed_data, events, sampling_rate=a[1], epochs_start=0,
                              epochs_end=10) # ends 10 as there might be some carryover
    results = nk.bio_analyze(epochs, sampling_rate=a[1], method='interval-related')
    return results

In [71]:
for file, ev_file in zip(acq_files, events_file):
    sub = file.split('kpe')[1].split('/')[0]
    print(sub)
    print(ev_file)
    result = readAnalyze_acq(file, ev_file)
    result['subject_id'] = sub
    result.to_csv(output_dir + '/sub-' + sub + '_ses-' + ses + '.csv', index=False, sep = '\t')

008
/media/Data/Lab_Projects/KPE_PTSD_Project/neuroimaging/KPE_BIDS/condition_files/sub-008_ses-1.csv
1223
/media/Data/Lab_Projects/KPE_PTSD_Project/neuroimaging/KPE_BIDS/condition_files/sub-1223_ses-1.csv


/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/neurokit2/eda/eda_intervalrelated.py:116: RuntimeWarning: invalid value encountered in double_scalars
  output["SCR_Peaks_Amplitude_Mean"] = np.nansum(amplitude) / np.sum(peaks)


1253
/media/Data/Lab_Projects/KPE_PTSD_Project/neuroimaging/KPE_BIDS/condition_files/sub-1253_ses-1.csv


/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/neurokit2/eda/eda_intervalrelated.py:116: RuntimeWarning: invalid value encountered in double_scalars
  output["SCR_Peaks_Amplitude_Mean"] = np.nansum(amplitude) / np.sum(peaks)


1263
/media/Data/Lab_Projects/KPE_PTSD_Project/neuroimaging/KPE_BIDS/condition_files/sub-1263_ses-1.csv
1293
/media/Data/Lab_Projects/KPE_PTSD_Project/neuroimaging/KPE_BIDS/condition_files/sub-1293_ses-1.csv


...

/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/neurokit2/eda/eda_intervalrelated.py:116: RuntimeWarning: invalid value encountered in double_scalars
  output["SCR_Peaks_Amplitude_Mean"] = np.nansum(amplitude) / np.sum(peaks)


1573
/media/Data/Lab_Projects/KPE_PTSD_Project/neuroimaging/KPE_BIDS/condition_files/sub-1573_ses-1.csv


/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/neurokit2/eda/eda_intervalrelated.py:116: RuntimeWarning: invalid value encountered in double_scalars
  output["SCR_Peaks_Amplitude_Mean"] = np.nansum(amplitude) / np.sum(peaks)


1578
/media/Data/Lab_Projects/KPE_PTSD_Project/neuroimaging/KPE_BIDS/condition_files/sub-1578_ses-1.csv


/home/or/miniconda3/envs/neuroAnalysis/lib/python3.7/site-packages/neurokit2/eda/eda_intervalrelated.py:116: RuntimeWarning: invalid value encountered in double_scalars
  output["SCR_Peaks_Amplitude_Mean"] = np.nansum(amplitude) / np.sum(peaks)


In [49]:
result

,SCR_Peaks_N,SCR_Peaks_Amplitude_Mean,subject_id
1,1.0,0.042608,1561
2,0.0,NaN,1561
3,1.0,0.007511,1561
4,1.0,0.047930,1561
5,1.0,0.003172,1561
6,1.0,0.044502,1561
7,1.0,0.289602,1561
8,1.0,0.147923,1561
9,1.0,0.200464,1561


In [50]:
condition_list = ['sad1','trauma1','relax1', 'sad2','trauma2','']

In [2]:
nk.cite()


You can cite NeuroKit2 as follows:

- Makowski, D., Pham, T., Lau, Z. J., Brammer, J. C., Lesspinasse, F., Pham, H.,
  Schölzel, C., & S H Chen, A. (2020). NeuroKit2: A Python Toolbox for Neurophysiological
  Signal Processing. Retrieved October 25, 2020, from https://github.com/neuropsychology/NeuroKit


Full bibtex reference:

@misc{neurokit2,
  doi = {10.5281/ZENODO.3597887},
  url = {https://github.com/neuropsychology/NeuroKit},
  author = {Makowski, Dominique and Pham, Tam and Lau, Zen J. and Brammer, Jan C. and Lesspinasse,
            Fran\c{c}ois and Pham, Hung and Schölzel, Christopher and S H Chen, Annabel},
  title = {NeuroKit2: A Python Toolbox for Neurophysiological Signal Processing},
  publisher = {Zenodo},
  month={Mar},
  year = {2020},
}

